In [ ]:
#!pip install pytesseract
#!pip install pypdf2
#!pip install pdf2image

!pip install tika
!pip install nltk


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import re
import spacy
import pandas as pd
from spacy.matcher import Matcher

In [ ]:
from tika import parser
resume = r'disini file.pdf'
resume_data = parser.from_file(resume)

text = resume_data['content']
print(text)

parsed_content = {}

In [ ]:
#e-mail
def email(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(string)

email = email(text)
print("e-mail:", email)
parsed_content['E-mail'] = email

#phone number
def phone_number(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(string)
    return [re.sub(r'\D', '', number) for number in phone_numbers]

phone_number = phone_number(text)
print("phone number:", phone_number)
parsed_content['Phone number'] = phone_number


In [ ]:
nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

def get_name(text):
   nlp_text = nlp(text)
   pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
  
   matcher.add('NAME', [pattern], on_match = None)
  
   matches = matcher(nlp_text)
  
   for match_id, start, end in matches:
       span = nlp_text[start:end]
       return span.text

name = get_name(text)
print("name/sector:", name)
parsed_content['Name'] =  name

In [ ]:
text = text.replace("\n"," ")
text = text.replace("[^a-zA-Z0-9]", " ");  
re.sub('\W+','', text)
text = text.lower()
print(text)

In [ ]:
Keywords = ["education","summary","personal profile",
            "work background","qualifications","experience",
            "skills","achievements","projects"]

content = {}
indices = []
keys = []
for key in Keywords:
    try:
        content[key] = text[text.index(key) + len(key):]
        indices.append(text.index(key))
        keys.append(key)
    except:
        pass
    
#sorting the index
zipped_lists = zip(indices, keys)
sorted_pairs = sorted(zipped_lists)
sorted_pairs

tuples = zip(*sorted_pairs)
indices, keys = [ list(tuple) for tuple in  tuples]
keys

In [ ]:
# menghapus bagian yang berlebihan
content = []
for idx in range(len(indices)):
    if idx != len(indices)-1:
        content.append(text[indices[idx]: indices[idx+1]])
    else:
        content.append(text[indices[idx]: ])

for i in range(len(indices)):
    parsed_content[keys[i]] = content[i]  

In [ ]:
parsed_content

In [ ]:
# print dalam bentuk df
#pd.DataFrame(parsed_content.items())


In [ ]:
import re

# regex
email_regex = r'[\w\.-]+@[\w\.-]+'
name_regex = r'Name: (.+)'
education_regex = r'education\s+(.+?)\s+experience'
experience_regex = r'experience\s+(.+?)\s+projects'
projects_regex = r'projects\s+(.+?)\s+certifications'
skills_regex = r'skills\s+(.+)'

# mengambil informasi dengan regex
email_matches = re.findall(email_regex, text)
name_match = re.search(name_regex, text)
education_match = re.search(education_regex, text, re.DOTALL)
experience_match = re.search(experience_regex, text, re.DOTALL)
projects_match = re.search(projects_regex, text, re.DOTALL)
skills_match = re.search(skills_regex, text)

# informasi yang diambil
parsed_content = {
    'E-mail': email_matches,
    'Name': name_match.group(1) if name_match else '',
    'education': education_match.group(1) if education_match else '',
    'experience': experience_match.group(1) if experience_match else '',
    'projects': projects_match.group(1) if projects_match else '',
    'skills': skills_match.group(1) if skills_match else ''
}

parsed_content


In [ ]:
# print dalam bentuk df
pd.DataFrame(parsed_content.items())


In [ ]:
def clean_resume(parsed_content):
    # memakai text parsed_content
    text = parsed_content.get('education', '') + ' ' + parsed_content.get('experience', '') + ' ' + parsed_content.get('projects', '') + ' ' + parsed_content.get('skills', '')

    # Remove URLs
    text = re.sub('http\S+\s*', ' ', text)
    # Remove RT and cc
    text = re.sub('RT|cc', ' ', text)
    # Remove hashtags
    text = re.sub('#\S+', '', text)
    # Remove mentions
    text = re.sub('@\S+', ' ', text)
    # Remove punctuations and non-ASCII characters
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text)
    
    return text

cleaned_text = clean_resume(parsed_content)
print(cleaned_text)


In [ ]:
def extract_skills(text):
    nlp_text = nlp(text)

    # menghapus stopwords dan melakukan token
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    skills = ["machine learning","deep learning","nlp","natural language processing","mysql","sql",
              "computer vision","tensorflow","opencv","python","matlab"]
   
    setskills = []
    for token in tokens:
        if token.lower() in skills:
            setskills.append(token)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            setskills.append(token)
    return [i.capitalize() for i in set([i.lower() for i in setskills])]

skills = []
skills = extract_skills(text)

parsed_content["Skills"] = skills
skills 

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')


In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # tokenisasi
    tokens = word_tokenize(text)
    # hapus stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # mengembalikan ke bentuk awal
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# terapkan stopwords
filtered_text = remove_stopwords(cleaned_text)
print(filtered_text)

In [ ]:
#df

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# inisialisasi stemmer dan lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_word(text):
    # tokenissasi teks
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # lemmatization
    lemmatized_words = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    filtered_lemma_text = ' '.join(lemmatized_words)
    return filtered_lemma_text

filtered_lemma_text = lemmatize_word(cleaned_text)
print("Lemmatized words:", filtered_lemma_text)


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

def tokenize_text(text):
    # tokenisasi
    tokens = word_tokenize(text)
    return tokens

tokenized_text = tokenize_text(filtered_lemma_text)
print("Tokenized words:\n", tokenized_text)